In [6]:
import pandas as pd
import re 
import os
# 1. Access the corresponding .txt, .wav, and .avi files for each EDA label

# Extract the conversation filename and speaker informationfrom the dataset
eda_df = pd.read_csv("eda_iemocap_no_utts_dataset.csv")
eda_df = eda_df[["speaker", "utt_id", "EDA"]]
filename_ids = []
speaker_M_F = []
session_numbers = []
for i, row in eda_df.iterrows():
    match = re.search(r"b'(Ses(\d+)[MF]_.+\d+.*)_([MF])", row["speaker"])
    filename_ids.append(match.group(1))
    session_numbers.append(int(match.group(2))) 
    speaker_M_F.append(match.group(3))
eda_df = eda_df.drop(columns=["speaker"])
eda_df["filename"] = filename_ids
eda_df["session_number"] = session_numbers
eda_df["speaker"] = speaker_M_F
eda_df
print(len(eda_df))
# Access transcipt files based on filename
utt_df = []
root_dir = "IEMOCAP_full_release/"
for i in range(1, 6):
    directory = os.path.join(root_dir, f"Session{i}/dialog/transcriptions/")
    for entry in os.scandir(directory):  
        if entry.is_file() and entry.path.endswith(".txt"):  # check if it's a file
            with open(entry.path, "r") as file:
                lines = file.readlines()
                for order, line in enumerate(lines):
                    speaker_info, utterance = line.split(":")[0], line.split(":")[1]
                    pattern = r"(F|M)(\d+)\s\[(\d+\.\d+)-(\d+\.\d+)\]"
                    match = re.search(pattern, speaker_info)
                    if match is None:
                        continue
                    speaker_f_m = match.group(1)
                    utt_id = match.group(2)
                    start = match.group(3)
                    end = match.group(4)
                    utt_df.append({"utt_id": int(utt_id),"start": float(start), "end": float(end), "speaker": speaker_f_m, "utterance": utterance.strip(), "session_number": int(i), "original_order": order})
utt_df = pd.DataFrame(utt_df)
utt_df.head(5)
# # Combine the EDA and utterances together
final_df = pd.merge(eda_df, utt_df, on=[["utt_id", "session_number", "speaker"]])
final_df


# # 2. Create a dataset following agreement estimation project for each modality or each utterance
# # 3. Run some training on 80% of the scripted data on the lab machine
# # 4. Determine the training/test splits (think of emotion distribution, scenarios, speaker)

10039


KeyError: ['utt_id', 'session_number', 'speaker']

In [ ]:
utt_id

In [4]:
eda_df

,speaker,utt_id,EDA
0,b'Ses01M_impro07_M',0,sd
1,b'Ses01M_impro07_F',0,b
2,b'Ses01M_impro07_M',1,sd
3,b'Ses01M_impro07_F',1,qy
4,b'Ses01M_impro07_M',2,qy
...,...,...,...
10034,b'Ses05F_script01_3_F',24,qy
10035,b'Ses05F_script01_3_M',39,sd
10036,b'Ses05F_script01_3_F',25,qy
10037,b'Ses05F_script01_3_M',40,sd


In [9]:
session = "Ses01M_impro07"
transcription = f"IEMOCAP_full_release/Session1/dialog/transcriptions/{session}.txt"
# video = f"IEMOCAP_full_release/Session1/dialog/avi/DivX/{session}.avi"

with open(transcription, "r") as file:
    lines = file.readlines()

lines

["Ses01M_impro07_M000 [002.6812-007.9800]: Check this out.  You know how I've told you I've been really into like softball recently?\n",
 'Ses01M_impro07_F000 [007.6300-008.5700]: Yeah.\n',
 'Ses01M_impro07_M001 [008.2200-014.7500]: Well, this is totally random, I got this full ride scholarship to go to U.S.C next year to play softball.\n',
 "Ses01M_impro07_F001 [013.9500-021.1200]: [LAUGHTER]. For softball? That's unbelievable. Is it slow pitch?\n",
 "Ses01M_impro07_M002 [015.5400-020.6700]: For softball.  They're going to pay me to go to school. Isn't that awesome?\n",
 "Ses01M_impro07_M003 [020.9200-034.3800]: Yeah.  Well, I don't--well yeah I can pitch, I'm not like the pitcher though. I'm like the pinch hitter guy I mostly just hit.  But I guess I'm going to have to... Yeah, and then I play shortstop too but... It's so ex- The good news is I get to stay here.\n",
 'Ses01M_impro07_F002 [023.8800-025.2500]: Uh huh.\n',
 'Ses01M_impro07_F003 [027.0600-029.8100]: Pinch hitting softbal